In [1]:
import sys, os
sys.path.append('/home/dnelson/project/msprime')
import msprime

import pandas as pd
import numpy as np
from collections import Counter
from tqdm import tqdm
from collections import defaultdict

In [8]:
df.head()

,ind,father,mother,sex
0,10086,0,0,1
1,10087,0,0,2
2,10102,0,0,1
3,10103,0,0,2
4,10128,0,0,1


In [359]:
class PedFiller:
    def __init__(self, ped_df):
        self.ped_df = ped_df
        self.min_ID = 0
        self.probands = set()
        self.new_ped = None
        # TODO: Would be good to have 'parent_cohorts' where
        # we can choose from pre-existing couples rather than
        # just random individuals from each cohort
        # - could build cohorts by collecting parents who had a
        #   child at the given time, that way the generation time
        #   is chosen automatically from the pedigree itself.
        self.node_cohorts = defaultdict(list)
        self.founder_cohorts = defaultdict(list)

    @staticmethod
    def read_ped(pedfile, header=0, sep=' ', dtype=int, usecols=(0, 1, 2)):
        df = pd.read_csv(pedfile, header=header, sep=sep, dtype=dtype, usecols=usecols)
        for expected_col in ['ind', 'mother', 'father']:
            assert(expected_col in df.columns)
        
        return PedFiller(df)

    def partial_order_times(self):
        ind_dict = dict(zip(self.ped_df['ind'], range(len(self.ped_df['ind']))))
        not_mothers = set(self.ped_df['ind']).difference(self.ped_df['mother'])
        self.probands = not_mothers.difference(self.ped_df['father'])

        climbers = self.probands
        times = np.zeros(len(self.ped_df['ind']))
        t = 0
        while len(climbers) > 0:
            next_climbers = set()
            for c in tqdm(climbers, leave=False, desc="Setting time {}".format(t)):
                idx = ind_dict[c]
                time = times[idx]
                if t > time:
                    times[idx] = t

                mother = self.ped_df['mother'][idx]
                father = self.ped_df['father'][idx]
                if mother != 0:
                    next_climbers.add(mother)
                if father != 0:
                    next_climbers.add(father)

            climbers = set(next_climbers)
            t += 1
        
        self.ped_df['time'] = times
        
    def build_cohorts(self):
        """
        Collects individuals into cohorts of identical times
        """
        for i, row in tqdm(self.ped_df.iterrows(),
                           total=self.ped_df.shape[0],
                           desc='Building cohorts'):
            if row.mother == 0 and row.father == 0:
                self.founder_cohorts[row.time].append(int(row.ind))

            # Node cohorts contain both founders and internal nodes
            self.node_cohorts[row.time].append((row.ind))
            
    def get_wf_parents(self, n, N, min_ID, monogamous=True):
        """
        Draws new parents for unconnected recent founders from a randomly-mating
        Wright-Fisher population
        """
        if monogamous is not True:
            raise NotImplementedError
            
        num_couples = N / 2
        mothers = np.arange(min_ID,
                            min_ID + num_couples,
                            dtype=int
                           ).reshape(-1, 1)
        fathers = np.arange(min_ID + num_couples,
                            min_ID + (num_couples * 2),
                            dtype=int
                           ).reshape(-1, 1)
        
        if monogamous is True:
            np.random.shuffle(fathers)
            couple_indices = np.random.randint(0, num_couples, size=n)
            couples = np.concatenate([mothers, fathers], axis=1)[couple_indices]
            
        return couples
    
    def complete_pedigree(self, N=100, reconnection_rate=0.05, max_gen_diff=1):
        """
        Draws new connections for recent founders from parents within max_gen_diff
        of the recent founder's time.
        """
        assert(len(self.founder_cohorts) > 0)
        assert(len(self.node_cohorts) > 0)
        
        max_time = int(np.max(self.ped_df['time']))
        max_ID = np.max(self.ped_df['ind'])

        self.new_ped_rows = []  
        for time in tqdm(range(max_time), desc='Drawing new connections'):
            founders = self.founder_cohorts[time]
            if len(set(founders)) != len(founders):
                print("Error at time", time)
                print(founders)
                break
            if len(founders) == 0:
                continue

            possible_parents = []
            for i in range(1, max_gen_diff + 1):
                possible_parents.extend(self.node_cohorts[time + i])
            assert(len(possible_parents) == len(set(possible_parents)))

            # TODO: Might be a problem here - in full QC pedigree seems too many
            #       reconnections drawn.
            num_to_reconnect = np.random.binomial(len(founders), reconnection_rate)
            np.random.shuffle(founders)
            to_reconnect = founders[:num_to_reconnect]
            not_reconnected = founders[num_to_reconnect:]
            
            new_parents = self.get_wf_parents(len(not_reconnected), N, max_ID)
            max_ID = np.max(new_parents)
            for founder, parents in zip(not_reconnected, new_parents):
                mother, father = parents
                row = [founder, mother, father, time]
                self.new_ped_rows.append(row)
                
            # Add new parents to next founder generation
            self.founder_cohorts[time + 1].extend(set(list(new_parents.ravel())))
                
            for founder in to_reconnect:
                mother, father = np.random.choice(possible_parents, size=2)
                row = [founder, mother, father, time]
                self.new_ped_rows.append(row)
                
    def collect_rows(self):
        """
        Sorts original and new individuals into unchanged/updated/new categories,
        then combines them into a new array
        """
        new_df = pd.DataFrame(self.new_ped_rows,
                              columns=['ind', 'father', 'mother', 'time']
                             ).set_index('ind').sort_index()
        original_df = self.ped_df.set_index('ind').sort_index()

        self.merged_rows = []
        self.unchanged_rows = []
        self.new_rows = []
        new_iter = new_df.iterrows()
        new_founder, new_row = next(new_iter)
        
        # Replace reconnected founder rows
        for ind, row in original_df.iterrows():
            if new_founder == ind:
                self.merged_rows.append(
                    [new_founder,
                     int(new_row.father),
                     int(new_row.mother),
                     new_row.time]
                )
                new_founder, new_row = next(new_iter)
            else:
                self.unchanged_rows.append(
                    [ind,
                     int(row.father),
                     int(row.mother),
                     row.time]
                )
                
        # Add new individuals
        for founder, row in new_iter:
            self.new_rows.append(
                [founder,
                 int(row.father),
                 int(row.mother),
                 row.time]
            )
            
        assert(len(self.unchanged_rows) + len(self.merged_rows) == self.ped_df.shape[0])
        
        self.new_ped = self.unchanged_rows + self.merged_rows + self.new_rows
            
    def write_filled_ped(self, path):
        with open(os.path.expanduser(path), 'w') as f:
            f.write('ind\tfather\tmother\ttime\n')
            for row in self.new_ped:
                f.write('\t'.join([str(x) for x in row]) + '\n')
                


In [360]:
P = PedFiller.read_ped('/Users/dnelson/project/anc_finder/data/BALasc_probands1930.txt', sep='\t')

P.partial_order_times()
P.build_cohorts()
P.complete_pedigree()
P.collect_rows()

# new_df = pd.DataFrame(P.new_ped_rows, columns=['ind', 'father', 'mother', 'time']).set_index('ind')
# original_df = P.ped_df.set_index('ind')

# original_founders = original_df[(original_df.mother == 0) & (original_df.father == 0)]
# original_nodes = original_df[(original_df.mother != 0) | (original_df.father != 0)]

# original_founders = original_founders.sort_index()
# new_df = new_df.sort_index()

Building cohorts: 100%|██████████| 3423179/3423179 [03:07<00:00, 18225.47it/s]


In [361]:
P.write_filled_ped('~/temp/filled_BALasc_probands1930.txt')

In [368]:
len(P.merged_rows)

354410

In [367]:
len(P.new_ped_rows)

356008

In [365]:
for k, v in P.founder_cohorts.items():
    print(k, len(v))

4.0 16087
3.0 30526
2.0 70156
1.0 210543
5.0 8520
6.0 3827
8.0 1844
7.0 2156
9.0 2162
12.0 1500
11.0 1486
10.0 1894
13.0 1928
14.0 2176
15.0 821
16.0 267
17.0 115
18.0 92
0 0


In [300]:
original_iter = original_founders.iterrows()
new_iter = new_df.iterrows()

merged_rows = []
unchanged_rows = []
founder, row = next(original_iter)
new_founder, new_row = next(new_iter)
while True:
    if new_founder == founder:
        merged_rows.append(new_founder)
        
        founder, row = next(original_iter)
        new_founder, new_row = next(new_iter)
    else:
        unchanged_rows.append([founder, row.father, row.mother, row.time])
        founder, row = next(original_iter)

    
    

StopIteration: 

In [301]:
unchanged_rows

[[18525, 0.0, 0.0, 17.0],
 [18526, 0.0, 0.0, 17.0],
 [18527, 0.0, 0.0, 17.0],
 [18528, 0.0, 0.0, 17.0],
 [18539, 0.0, 0.0, 17.0],
 [18540, 0.0, 0.0, 17.0],
 [65396, 0.0, 0.0, 17.0],
 [65397, 0.0, 0.0, 17.0],
 [76422, 0.0, 0.0, 17.0],
 [76423, 0.0, 0.0, 17.0]]

In [302]:
len(merged_rows)

7389

In [303]:
P.founder_cohorts[17]

[18525,
 18526,
 18527,
 18528,
 18539,
 18540,
 65396,
 65397,
 76422,
 76423,
 901891,
 901892,
 901893,
 901894,
 901895,
 901896,
 901897,
 901898,
 901899,
 901900,
 901901,
 901902,
 901903,
 901904,
 901905,
 901906,
 901907,
 901908,
 901909,
 901910,
 901911,
 901912,
 901914,
 901915,
 901916,
 901917,
 901919,
 901920,
 901921,
 901922,
 901923,
 901924,
 901925,
 901926,
 901927,
 901928,
 901929,
 901930,
 901931,
 901932,
 901933,
 901934,
 901935,
 901936,
 901937,
 901938,
 901939,
 901940,
 901941,
 901942,
 901943,
 901944,
 901945,
 901946,
 901947,
 901948,
 901949,
 901950,
 901951,
 901952,
 901953,
 901954,
 901955,
 901956,
 901957,
 901958,
 901959,
 901961,
 901962,
 901963,
 901964,
 901965,
 901966,
 901967,
 901968,
 901969,
 901970,
 901971,
 901973,
 901974,
 901975,
 901976,
 901977,
 901978,
 901979,
 901980,
 901981,
 901982,
 901983,
 901984,
 901985,
 901986,
 901987,
 901988,
 901989,
 901990]

In [76]:
original_founders.head()

,father,mother,sex,time
ind,,,,
10086,0,0,1,9.0
10087,0,0,2,9.0
10102,0,0,1,7.0
10103,0,0,2,7.0
10128,0,0,1,8.0


In [77]:
new_df.head()

,father,mother,time
ind,,,
10086,901211.0,901250.0,9
10087,901195.0,901241.0,9
10102,900996.0,901055.0,7
10103,901004.0,901077.0,7
10128,901101.0,901170.0,8


In [70]:
next(a)

(10086, father    0.0
 mother    0.0
 sex       1.0
 time      9.0
 Name: 10086, dtype: float64)

In [47]:
new_df.head()

,father,mother,time
ind,,,
863186,900554.0,900594.0,2
861893,900529.0,900572.0,2
136980,900551.0,900588.0,2
861894,900514.0,900556.0,2
454426,900510.0,900574.0,2


In [50]:
10086 in new_df.index

True

In [26]:
P.ped_df.head()

,ind,father,mother,sex,time
0,10086,0,0,1,9.0
1,10087,0,0,2,9.0
2,10102,0,0,1,7.0
3,10103,0,0,2,7.0
4,10128,0,0,1,8.0


In [15]:
cohorts = {}
for i in range(20):
    cohorts[i] = df['mother'][(df['cohort'] == i) & (df['mother'] != 0)]

In [16]:
[len(v) for v in cohorts.values()]

[1504150,
 615427,
 339214,
 215890,
 143180,
 93538,
 59021,
 37927,
 24690,
 15764,
 9445,
 5384,
 2989,
 1585,
 457,
 96,
 9,
 1,
 0,
 0]

In [17]:
naive_partner = dict(zip(df['mother'], df['father']))

In [18]:
naive_partner[8232207]

1868793

In [19]:
max_time = 12
ped_list = []
num_assigned = 0
for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    mother = row.mother
    father = row.father
    if row.mother == 0 and row.cohort <= max_time:
        num_assigned += 1
        parent_choices = np.concatenate([cohorts[t] for t in range(int(row['first']), int(row['last'])+1)])
        try:
            mother = np.random.choice(parent_choices)
        except:
            print(i)
            print(row)
            raise
        father = naive_partner[mother]
        
    ped_list.append([row.ind, mother, father])
    

100%|██████████| 3423179/3423179 [06:56<00:00, 8221.20it/s] 


In [20]:
num_assigned

349601

In [21]:
with open('/Users/dnelson/project/pedigree_msp/data/BALSAC/BALasc_probands1930_12gens_reconnected.txt', 'w') as f:
    f.write('ind\tfather\tmother\n')
    for row in tqdm(ped_list, total=len(ped_list)):
        f.write('\t'.join([str(x) for x in row]) + '\n')

100%|██████████| 3423179/3423179 [00:08<00:00, 394751.93it/s]


In [22]:
[r for r in ped_list if 2331548 in r]

[[2340939, 2331548, 2331549],
 [2750947, 2331548, 2385356],
 [2331548, 8232298, 2381228]]

In [23]:
[r for r in ped_list if 1104914 in r]

[[1104914, 2333034, 2333035], [1104916, 1104914, 1104915]]

In [25]:
Counter(df['cohort'].values)

Counter({0: 1504150,
         1: 825970,
         2: 409270,
         3: 246316,
         4: 159167,
         5: 101958,
         6: 62748,
         7: 39983,
         8: 26434,
         9: 17826,
         10: 11239,
         11: 6770,
         12: 4389,
         13: 3413,
         14: 2533,
         15: 817,
         16: 176,
         17: 18,
         18: 2})